<a href="https://colab.research.google.com/github/felp99/PythonFinancas/blob/main/%5BYOUTUBE%5D_Compare_a_SELIC_com_qualquer_investimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBOV vs SELIC: Qual melhor investimento? Aula 1

## Felipe LOBATO
Contato para dúvidas

- **Instagram:** https://www.instagram.com/felp99/
- **Linkedin:** https://www.linkedin.com/in/felipelobato99/
- **E-mail:** felipelobato99@gmail.com

## Requisição de Bibliotecas

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


## Requisição dos dados

In [ ]:
# API Banco do Brasil para SELIC
res_selic = requests.get('https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json')

#Biblioteca yfinance para IBOV
res_ibov = yf.Ticker('Gold')

## Tratamento dados SELIC

In [ ]:
# Transforma em Dataframe
selic_df = pd.DataFrame(res_selic.json())

# Coluna 'data' para formato correto
selic_df['data'] = pd.to_datetime(selic_df['data'], format='%Y/%m/%d', errors='ignore')

#Coluna 'valor' para formato correto
selic_df['SELIC'] = selic_df['valor'].astype(float)

#Dropa coluna 'valor' antiga
selic_df.drop(columns=['valor'], inplace = True)

#Seta o index para ser a nova coluna 'data' com formato correto
selic_df.set_index('data', inplace = True)

#Mostra as 10 primeiras linhas
selic_df.head(10)

,SELIC
data,
04/06/1986,0.065041
05/06/1986,0.067397
06/06/1986,0.066740
09/06/1986,0.068247
10/06/1986,0.067041
11/06/1986,0.066740
12/06/1986,0.066575
13/06/1986,0.066164
16/06/1986,0.066493


## Tratamento dados OUTRO ATIVO (IBOV)

In [ ]:
# Seta a data inicial e final para os dados
ibov_df = res_ibov.history(start="1986-01-01", end=None)

# Retira colunas não utilizadas
ibov_df.drop(columns=['Open', 'High', 'Low',	'Volume',	'Dividends',	'Stock Splits'], inplace=True)

# Renomeia coluna 'Close' para 'IBOV' (Valor de fechamento)
ibov_df.rename(columns={'Close':'IBOV'}, inplace=True)

#Seta o formato correto do index
ibov_df.index = ibov_df.index.strftime('%d/%m/%Y')

ibov_df

,IBOV
Date,
02/01/1986,0.546214
03/01/1986,0.546214
06/01/1986,0.546214
07/01/1986,0.546214
08/01/1986,0.546214
...,...
08/02/2024,14.820000
09/02/2024,14.670000
12/02/2024,14.730000


## Definindo data de início e fim

In [ ]:
# Junta os dois dataframes (inner join)
df = pd.merge(ibov_df, selic_df, left_index=True, right_index=True)

#Define data inicial de análise
init = '18/01/2022'

#Define data final de análise
final = None

#Corta o dataframe de acordo com as datas e o index
df = df[init : ] #Colocar data final se você a utilizar

#Mostra o dataframe
df.head()

,IBOV,SELIC
18/01/2022,17.465096,0.034749
19/01/2022,18.980120,0.034749
20/01/2022,18.735456,0.034749
21/01/2022,18.199080,0.034749
24/01/2022,18.227312,0.034749


## Lógica de crescimento

### IBOV
IBOV está em valor bruto, calculamos o percentual de mudança mais 1.

### SELIC
SELIC está em percentual, dividimos por 100 e somamos 1.

### Soma 1 ao final

A soma do 1 nos percentuais significa que estamos sempre trabalhando com o valor acumulado das variações. Esse 1 seria equivalemente ao valor anterior mais (ou menos) a variação que acontece.

In [ ]:
# IBOV está em valor bruto
df['IBOV'] = df['IBOV'].pct_change() + 1

# SELIC em percentual
df['SELIC'] = (df['SELIC']/100) + 1

# Mostra o dataframe
df.head(10)

,IBOV,SELIC
18/01/2022,NaN,1.000347
19/01/2022,1.086746,1.000347
20/01/2022,0.987109,1.000347
21/01/2022,0.971371,1.000347
24/01/2022,1.001551,1.000347
25/01/2022,1.006195,1.000347
26/01/2022,0.976398,1.000347
27/01/2022,0.983184,1.000347
28/01/2022,0.991983,1.000347
31/01/2022,1.031789,1.000347


## Retorno Acumulado

Para calcular o retorno de taxas de um conjunto de taxas de retorno, você pode usar a fórmula do produto acumulado. O retorno total acumulado pode ser calculado multiplicando-se as taxas de retorno individuais de cada período, considerando que as taxas de retorno são expressas em forma decimal.

A fórmula para calcular o produto acumulado dos retornos $(R_{total})$ de $ n $ períodos é:

$$
R_{total} = (1 + r_1) \times (1 + r_2) \times \cdots \times (1 + r_n)
$$

onde:
- $R_{total}$ é o retorno total acumulado,
- $r_1, r_2, \ldots, r_n \$ são as taxas de retorno para cada período, expressas em forma decimal.


Após calcular o produto acumulado, para encontrar o retorno percentual total, você subtrai 1 do resultado e multiplica por 100:

$$
\text{Retorno Percentual Total} = (R_{total} - 1) \times 100
$$

Essa abordagem leva em consideração o efeito de composição, que é crucial para calcular o retorno de investimentos ao longo do tempo.

In [ ]:
# Produtório das taxas (que já tem o 1 somado)
df_cumprod = df.cumprod()

## Gráfico de Série

In [ ]:
# Definindo investimento inicial como uma constante (feito apenas uma vez)
INVESTIMENTO_INICIAL = 1

# Plota o gráfico
fig = px.line(df_cumprod * INVESTIMENTO_INICIAL, title='SELIC vs GOLD')
fig.show()

## Gráfico de Correlação
Um gráfico de correlação entre dois ativos, como o IBOVESPA (IBOV) e a taxa SELIC, durante um determinado período, visa ilustrar a relação existente entre os movimentos de preços ou retornos desses dois ativos ao longo do tempo. A correlação é uma medida estatística que descreve o grau de relação linear entre duas variáveis. Seus valores variam entre -1 e 1, onde:

- **1** indica uma correlação perfeitamente positiva, significando que os ativos se movem na mesma direção e na mesma proporção.
- **-1** indica uma correlação perfeitamente negativa, significando que os ativos se movem em direções opostas e na mesma proporção.
- **0** indica nenhuma correlação linear, significando que os movimentos de um ativo não têm relação previsível com os movimentos do outro.

In [ ]:
# Gráfico de correlação entre os ativos no período
fig = px.imshow(df_cumprod.corr())
fig.show()